<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/FaceFusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Step 1: Install Dependencies**
import os

# Set CUDA paths FIRST (before any installations)
os.environ["CUDA_PATH"] = "/usr/local/cuda"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64:/usr/lib/x86_64-linux-gnu"
os.environ["PATH"] += ":/usr/local/cuda/bin"

# Clone repository
!git clone -q https://github.com/qsardor/Deepfake roop > /dev/null 2>&1
%cd roop

# Fix dependency conflicts
!sed -i 's/numba==0.61.0/numba==0.59.1/g' requirements.txt  # Resolve cuDF/cuML conflict [[4]]
!sed -i 's/protobuf==4.23.4/protobuf==4.25.2/g' requirements.txt  # Align with TensorFlow [[4]]

# Install dependencies
!pip install --quiet --no-warn-conflicts --extra-index-url https://download.pytorch.org/whl/cu118 -r requirements.txt --force-reinstall > /dev/null 2>&1

# Cleanup conflicting manual CUDA installations
!apt-get remove --purge -y cuda-11-8 libcudnn8 libcudnn8-dev > /dev/null 2>&1  # Remove manual CUDA [[3]]

print("Installation done. Please restart the runtime (Runtime > Restart Runtime) and proceed to job steps.")

In [ ]:
#@title **Run Job (Google Drive)**
import os
from google.colab import drive, files
from IPython.display import display, Image

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define paths
DRIVE_DIR = "/content/drive/MyDrive/Deepfake"
INPUT_DIR = os.path.join(DRIVE_DIR, "input")
OUTPUT_DIR = os.path.join(DRIVE_DIR, "output")
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Check/Upload files
source_path = os.path.join(INPUT_DIR, "source.png")
target_files = [f for f in os.listdir(INPUT_DIR) if f.startswith("target.")]

if not os.path.exists(source_path):
    print("Uploading source.png...")
    files.upload()
if not target_files:
    print("Uploading target file (image/video)...")
    files.upload()
    target_files = [f for f in os.listdir(INPUT_DIR) if f.startswith("target.")]

# Detect target type
target_file = os.path.join(INPUT_DIR, target_files[0])
output_ext = "png" if target_file.endswith(".png") else "mp4"
output_path = os.path.join(OUTPUT_DIR, f"result.{output_ext}")

# Set CUDA environment variables
os.environ["CUDA_PATH"] = "/usr/local/cuda"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64:/usr/lib/x86_64-linux-gnu"
os.environ["PATH"] += ":/usr/local/cuda/bin"

# Run processing
%cd /content/roop
!python run.py -s "{source_path}" -t "{target_file}" -o "{output_path}" --frame-processor face_swapper face_enhancer --keep-frames --keep-fps

# Show result
if os.path.exists(output_path):
    if output_ext == "png":
        display(Image(output_path))
    else:
        print(f"Video saved to {output_path}")
    print(f"Result saved to: {output_path}")
else:
    print("Processing failed. Check input files and GPU runtime.")

In [ ]:
#@title **Step 2: Run Job (Google Drive)**
import os
from google.colab import drive
from IPython.display import display, Image

# Set CUDA paths AGAIN after restart
os.environ["CUDA_PATH"] = "/usr/local/cuda"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64:/usr/lib/x86_64-linux-gnu"
os.environ["PATH"] += ":/usr/local/cuda/bin"

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define paths
DRIVE_DIR = "/content/drive/MyDrive/Deepfake"
INPUT_DIR = os.path.join(DRIVE_DIR, "input")
OUTPUT_DIR = os.path.join(DRIVE_DIR, "output")
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Check files
if not os.path.exists(os.path.join(INPUT_DIR, "source.png")):
    raise FileNotFoundError("Missing source.png in Google Drive input folder")
if not os.path.exists(os.path.join(INPUT_DIR, "target.mp4")):
    raise FileNotFoundError("Missing target.mp4 in Google Drive input folder")

# Run processing
%cd /content/roop
!python run.py -s "{os.path.join(INPUT_DIR, 'source.png')}" \
               -t "{os.path.join(INPUT_DIR, 'target.mp4')}" \
               -o "{os.path.join(OUTPUT_DIR, 'result.mp4')}" \
               --frame-processor face_swapper face_enhancer \
               --keep-frames --keep-fps

# Show result
if os.path.exists(os.path.join(OUTPUT_DIR, "result.mp4")):
    print(f"Video saved to: {OUTPUT_DIR}/result.mp4")
else:
    print("Processing failed. Check CUDA configuration.")